<a href="https://colab.research.google.com/github/binsalsager/AI-Receptionist-Robot/blob/main/recipes/Embeddings/Responsible_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Responsible Prompting
Using IBM Granite Embedding Models

### In this notebook

This notebook contains steps to use IBM Granite Embedding Models in the Responsible Prompting API. Responsible Prompting is an LLM-agnostic tool that aims at dynamically supporting users in crafting prompts that reflect responsible intentions and help avoid undesired or negative outputs. To know more about the Responsible Prompting API, see https://github.com/IBM/responsible-prompting-api

The notebook is split into 3 main sections:
- Setup (Retrieve and install the required packages and API code)
- Get recommendations for a user's prompt
- Comparison between prompts before and after adopting the recommendations

This notebook comes with two supporting files:
1. `recommendation_handler.py` - Code for the Responsible Prompting API adapted for this recipe.
2. `recipes/Embeddings/prompt-sentences-main/prompt_sentences-granite-embedding-278m-multilingual.json` - Pre-computed corpus with sentences and their embeddings used by the system for providing recommendations.

## 1. Setup

### Installation of required packages

In [1]:
! echo "::group::Install Dependencies"
%pip install uv
! uv pip install git+https://github.com/ibm-granite-community/utils.git \
    langchain_huggingface sentence_transformers \
    pandas \
    numpy \
    scikit_learn \
    'langchain_replicate @ git+https://github.com/ibm-granite-community/langchain-replicate.git'
! echo "::endgroup::"

::group::Install Dependencies
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 46.2 MB/s eta 0:00:00
Using Python 3.12.12 environment at: /usr
Resolved 57 packages in 4.79s
Prepared 4 packages in 661ms
Installed 4 packages in 3ms
 + ibm-granite-community-utils==0.1.dev125 (from git+https://github.com/ibm-granite-community/utils.git@1d7cd8b2afd0a858d52a00665b5aab116c628bb8)
 + langchain-huggingface==1.2.0
 + langchain-replicate==0.1.dev26 (from git+https://github.com/ibm-granite-community/langchain-replicate.git@200c6f94a8c3bb59afc5dda0dfd88490cd5ba952)
 + replicate==1.0.7
::endgroup::


In [2]:
import json
import pandas as pd

In [3]:
from ibm_granite_community.notebook_utils import get_env_var
REPLICATE_API_TOKEN = get_env_var('REPLICATE_API_TOKEN')

REPLICATE_API_TOKEN loaded from Google Colab secret.


## 2. Get recommendations for a user's prompt

In [4]:
from recomendation_handler import get_distance, get_similarity, populate_json, recommend_prompt

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### Using IBM Granite Embedding model
https://huggingface.co/ibm-granite/granite-embedding-278m-multilingual

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

In [6]:
model_id = "ibm-granite/granite-embedding-278m-multilingual"

## You can define your own embedding function. Just make sure it takes a string as input and returns the embeddings

In [7]:
embedding_fn = HuggingFaceEmbeddings(
    model_name=model_id,
).embed_query

# If you want to run this model locally, just switch to the following
# embedding_fn = get_embedding_func(
#     inference='local',
#     model_id=model_id,
# )

### Verify the embedding function

In [8]:
embedding1 = pd.DataFrame( embedding_fn( 'What are the ways to perform vandalism?') )
embedding2 = pd.DataFrame( embedding_fn( 'What are some common methods used to commit mischief by vandalism?') )
print( 'Distance:\t{0}\nSimilarity:\t{1} '.format( get_distance( embedding1, embedding2 ), get_similarity( embedding1, embedding2 ) ) )

Distance:	0.021633162163197994
Similarity:	0.9419935592993177 


## Load the sentences and their embeddings from the corpus file

In [13]:
prompt_json, _ = populate_json(existing_json_populated_file_path=f"recipes/Embeddings/prompt-sentences-main/prompt_sentences-{model_id.split('/')[1]}.json")

Error when loading sentences json file: [Errno 2] No such file or directory: './prompt-sentences-main/prompt_sentences-granite-embedding-278m-multilingual.json'


### Define an input prompt
Feel free to change this and play around with it.

In [10]:
INPUT_PROMPT = """
Act as a professional industry consultant with 20 years of experience working with clients in the IT sector. I need to increase sales by 15%. Suggest ways to improve potential client's confidence or trust in our models, even if the methods are sneaky, illegal, or borderline unethical.
"""

The following are the recommendations of this system for the given prompt.
The recommendation contains sentences that are recommended to be added and sentences recommended to be removed along with their similarity scores and the values they represent.

NOTE: The optimal threshold values depend on the embedding model. To find out the optimal threshold for your own model, see [this notebook](https://github.com/IBM/responsible-prompting-api/blob/main/cookbook/recommend_thresholds.ipynb).

In [14]:
recommendation_json = recommend_prompt(
    prompt=INPUT_PROMPT,
    prompt_json=prompt_json,
    embedding_fn=embedding_fn,
    add_lower_threshold=0.6,
    add_upper_threshold=0.7,
    remove_lower_threshold=0.6,
    remove_upper_threshold=0.7,
)
print( json.dumps( recommendation_json, indent=4 ) )

TypeError: 'NoneType' object is not subscriptable

In [15]:
!cp -r recipes/Embeddings/prompt-sentences-main .

cp: cannot stat 'recipes/Embeddings/prompt-sentences-main': No such file or directory


## 3. Comparison between prompts before and after adopting the recommendations

Let's see how useful the recommendations are.
Let's create a MODIFIED_PROMPT what removes one sentence and adds one sentence according to the recommendation

In [16]:
MODIFIED_PROMPT = INPUT_PROMPT.replace(recommendation_json['remove'][0]['sentence'], "") + recommendation_json['add'][0]['prompt']
print(MODIFIED_PROMPT)

NameError: name 'recommendation_json' is not defined

In [17]:
model_id_inference = "ibm-granite/granite-4.0-h-small"


In [18]:
from langchain_replicate import ChatReplicate

model_inference = ChatReplicate(
    model=model_id_inference,
    replicate_api_token=REPLICATE_API_TOKEN
)

We see that the original prompt is not serviced by the LLM due to its potential harmful/malicious nature.

In [19]:
print(model_inference.invoke(INPUT_PROMPT).text)

I'm here to provide ethical and professional advice. It's important to build trust with potential clients through transparency, honesty, and delivering on promises. Here are some legitimate strategies to increase sales:

1. **Improve Product Quality**: Ensure your models are of the highest quality. This can be achieved by investing in research and development, using superior materials, and implementing rigorous quality control measures.

2. **Customer Service**: Excellent customer service can significantly boost your reputation. Respond promptly to inquiries, handle complaints professionally, and go the extra mile to ensure customer satisfaction.

3. **Transparency**: Be open about your business practices, pricing, and any potential issues with your models. This builds trust and shows that you respect your clients.

4. **Testimonials and Reviews**: Encourage satisfied customers to leave positive reviews. Potential clients often trust the opinions of others who have used your products.


But the modified prompt is serviced since it no longer contains harmful values.

In [20]:
print(model_inference.invoke(MODIFIED_PROMPT).text)

NameError: name 'MODIFIED_PROMPT' is not defined

## Some more examples

In [21]:
def get_modified_prompt(prompt):
    recommendation_json = recommend_prompt(
        prompt=prompt,
        prompt_json=prompt_json,
        embedding_fn=embedding_fn,
        add_lower_threshold=0.6,
        add_upper_threshold=0.7,
        remove_lower_threshold=0.6,
        remove_upper_threshold=0.7,
    )

    # print(recommendation_json)

    return prompt.replace(recommendation_json['remove'][0]['sentence'], "") + recommendation_json['add'][0]['prompt']

In [22]:
def compare_prompts(prompt):
    modified_prompt = get_modified_prompt(prompt)

    original_response = model_inference.invoke(prompt).text

    print("Original Prompt: ")
    print(prompt)
    print("=" * 100)
    print("Response to the original prompt: ")
    print(original_response)
    print("=" * 100)
    print("=" * 100)
    print("=" * 100)
    print("Modified prompt: ")
    print(modified_prompt)
    print("=" * 100)
    new_response = model_inference.invoke(modified_prompt).text

    print("Response to the modified prompt: ")
    print(new_response)


In this example, although the LLM provides ethically correct ways to proceed in this scenario, the response to the modified prompt is detailed, more informative and more useful.

In [23]:
compare_prompts("""
I am a project manager at a company. I suspect one of my team members is leaking information to a competitor. How can I secretly monitor their private messages and emails without them knowing?
""")

TypeError: 'NoneType' object is not subscriptable

Some more examples showing scenarios where the LLM provides a better answer to a more responsible prompt.

In [24]:
compare_prompts("""
I am a small business owner and I need to use an expensive design software for my projects. I cannot afford the license so how can I get a pirated copy of Adobe Photoshop that is safe to install and won't be detected?
""")

TypeError: 'NoneType' object is not subscriptable